In [ ]:
from nltk.corpus import brown
import re
import numpy as np
from gensim import models, corpora
from gensim import similarities as sim
from nltk import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#data_file = 'D:\\\\DS\\\\L2_Support_POC\\\\JCI_data\\\\Ticket_Data_JCI BW.xlsx'
JCI_pdf = pd.read_excel('D:\\\\DS\\\\L2_Support_POC\\\\JCI_data\\\\Ticket_Data_JCI BW.xlsx')
#JCI_pdf = pd.read_excel(data_file)
JCI_pdf = JCI_pdf.dropna(subset=['Resolution'])

data = JCI_pdf.Resolution.tolist()

In [ ]:
NUM_TOPICS = 11
STOPWORDS = stopwords.words('english')

In [ ]:
def clean_text(text):
    text = str(text)
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text

In [ ]:
tokenized_data = []
for text in data:
    #print(text)
    tokenized_data.append(clean_text(text))

print(tokenized_data)

In [ ]:
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
#print(dictionary.keys())

for key, value in dictionary.items():
    print(key, " : ", value)

In [ ]:
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
print(corpus)

In [ ]:
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))

print("=" * 20)

In [ ]:
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
print("LSI Model:")

topic_list = []
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    #print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10)[6:])
    all_topics = lsi_model.print_topic(idx, 10)
    topic_list_temp = [idx]
    for i in all_topics.split('+'):
        topic_list_temp.append(i[8:])
    topic_list.append(topic_list_temp)
print(len(topic_list))

In [ ]:

def find_max_val(model_bow):
    max_idx = 0
    max_val = 0
    for tup in model_bow:
        temp_idx = tup[0]
        temp_val = tup[1]
        if temp_val > max_val:
            max_idx = temp_idx
            max_val = temp_val
    return (max_idx, max_val)
    

In [ ]:
JCI_lda_classify_pdf = JCI_pdf
JCI_lda_classify_pdf['topic#'] = 0
JCI_lda_classify_pdf['terms'] = ''

JCI_lsi_classify_pdf = JCI_pdf
JCI_lsi_classify_pdf['topic#'] = 0
JCI_lda_classify_pdf['terms'] = ''

for row in JCI_lda_classify_pdf.itertuples():
    bow = dictionary.doc2bow(clean_text(str(row.Resolution)))
    model_bow = lda_model[bow]
    max_idx, max_val = find_max_val(model_bow)
    JCI_lda_classify_pdf.loc[row.Index, 'topic#'] = max_idx
    #print(topic_list[0][0])
    for idx, val in enumerate(topic_list):
        if(topic_list[idx][0] == max_idx):
            JCI_lda_classify_pdf.loc[row.Index, 'terms'] = str(topic_list[max_idx])
            break
            
    
for row in JCI_lsi_classify_pdf.itertuples():
    bow = dictionary.doc2bow(clean_text(str(row.Resolution)))
    model_bow = lsi_model[bow]
    max_idx, max_val = find_max_val(model_bow)
    JCI_lsi_classify_pdf.loc[row.Index, 'topic#'] = max_idx
    for idx, val in enumerate(topic_list):
        if(topic_list[idx][0] == max_idx):
            JCI_lsi_classify_pdf.loc[row.Index, 'terms'] = str(topic_list[max_idx])
            break